In [83]:
from aocd import submit
import numpy as np
import re
import ast
from math import floor, ceil
from functools import reduce

In [247]:
YYYY=2021
DD=18

In [252]:
def load(filename):
    with open(filename,'r') as f:
         lines = [ast.literal_eval(s.rstrip("\n"))  for s in f]
    return lines
test1 = load("test1.dat")
test2 = load("test2.dat")
test3 = load("test3.dat")
test1

[[1, 2],
 [[1, 2], 3],
 [9, [8, 7]],
 [[1, 9], [8, 5]],
 [[[[1, 2], [3, 4]], [[5, 6], [7, 8]]], 9],
 [[[9, [3, 8]], [[0, 9], 6]], [[[3, 7], [4, 9]], 3]],
 [[[[1, 3], [5, 3]], [[1, 3], [8, 7]]], [[[4, 9], [6, 9]], [[8, 2], [7, 3]]]]]

In [7]:
data_in = load("input.dat")

# Teil a

In [407]:
class Snail:
    def __init__(self, val):
        if isinstance(val, Snail):
            if val.type == 0:
                self.set_number(val.val)
            else:
                self.left = val.left
                self.right = val.right
        if isinstance(val, list): 
            self.type = 1
            left, right = val
            self.left = Snail(left)
            self.right = Snail(right)
        else:
            self.set_number(val)
    
    def set_number(self, n):
        self.type = 0
        self.val = n
        self.left = None
        self.right = None
    
    def add_number(self, n):
        self.val += n
    
    def add_snail(self,s):
        self.left = self
        self.right = s
        self.type = 1
        
    def get_pair(self):
        return self.left, self.right
    
    def split(self):
        if self.type != 0 or self.val < 10:
            return
        self.type = 1
        self.left = Snail(floor(self.val/2))
        self.right = Snail(ceil(self.val/2))
    
    def __str__(self):
        if self.type == 0:
            return str(self.val)
        return f"[{self.left},{self.right}]"
    
    def to_list(self):
        if self.type == 0:
            return self.val
        return [self.left.to_list(), self.right.to_list()]
        

def explode(snail):
    explode_snail, left, right = e(snail)
    if explode_snail:
        if left:
            left.val+=explode_snail.left.val
        if right:
            right.val+=explode_snail.right.val
        explode_snail.set_number(0)
    
def e(snail, explode_snail=None, last_val_snail=None, level=0):
    if snail.type == 0:
        if explode_snail:
            return explode_snail, last_val_snail, snail
        else:
            return None, snail, None #last_val_snail = snail
    else:
        if level == 4 and not explode_snail:
            return snail, last_val_snail, None
        
        left, right = snail.get_pair()
        explode_snail, last_val_snail, next_val_snail  = e(left, explode_snail, last_val_snail, level+1)
        if not next_val_snail:
            explode_snail, last_val_snail, next_val_snail  = e(right, explode_snail, last_val_snail, level+1)
    
    return explode_snail, last_val_snail, next_val_snail

def split(snail):
    if snail.type == 0:
        #print(snail)
        if snail.val > 9:
            snail.split()
            return True
        return False
    left, right = snail.get_pair()
    if split(left):
        return True
    return split(right)

        
def snail_reduce(s):
    changed = True
    while changed:
        tmp = f"{s}"
        explode(s)
        if tmp != f"{s}":
            continue
        split(s)
        if tmp != f"{s}":
            continue
        changed = False
    return s

def magnitude(s):
    if s.type == 0:
        return s.val
    left, right = s.get_pair()
    return 3*magnitude(left)+2*magnitude(right)

    

In [408]:
def solution_a(data):
    snails = [Snail(s) for s in data]
    s1 = None
    for ss in snails:
        if not s1:
            s1 = snail_reduce(ss)
            #print(s1)
        else:
            s2 = snail_reduce(ss)
            #print(s2)
            s1 = snail_reduce(Snail([s1.to_list(), s2.to_list()]))
            #print(s1)
            #break
    return magnitude(s1)

In [410]:
print(solution_a(test3))

4140


In [401]:
submit(solution_a(data_in),part="a",day=DD, year=YYYY)

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


<Response [200]>

# Teil b

In [411]:
def solution_b(data):
    snails = [Snail(s) for s in data]
    m = 0
    for i, s1 in enumerate(data):
        for j, s2 in enumerate(data):
            if i==j:
                continue
            s = Snail([s1,s2])
            snail_reduce(s)
            mag = magnitude(s)
            if mag > m:
                m=mag
    return m

In [412]:
solution_b(test3)

3993

In [405]:
submit(solution_b(data_in),part="b",day=DD, year=YYYY)

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 18! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


<Response [200]>